In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import files
uploaded = files.upload()

Saving spam_ham_dataset.csv to spam_ham_dataset.csv


In [2]:
import string # to filer out pucntuations

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
df = pd.read_csv('spam_ham_dataset.csv')
df

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [4]:
df['text'] =df['text'].apply(lambda x: x.replace('\r\n', '')) # i am removing new row new line characters
df

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291this...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001( see att...",0
2,3624,ham,"Subject: neon retreatho ho ho , we ' re around...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springsthis deal is to bo...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ftthe transport vol...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following nomshpl ca...,0
5168,2933,ham,"Subject: calpine daily gas nomination>>julie ,...",0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [5]:
df.text.iloc[2] # 2nd row of text column

"Subject: neon retreatho ho ho , we ' re around to that most wonderful time of the year - - - neon leaders retreat time !i know that this time of year is extremely hectic , and that it ' s tough to think about anything past the holidays , but life does go on past the week of december 25 through january 1 , and that ' s what i ' d like you to think about for a minute .on the calender that i handed out at the beginning of the fall semester , the retreat was scheduled for the weekend of january 5 - 6 . but because of a youth ministers conference that brad and dustin are connected with that week , we ' re going to change the date to the following weekend , january 12 - 13 . now comes the part you need to think about .i think we all agree that it ' s important for us to get together and have some time to recharge our batteries before we get to far into the spring semester , but it can be a lot of trouble and difficult for us to get away without kids , etc . so , brad came up with a potentia

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


Preprocessing

In [ ]:
# remove puncutaion, all words to lower case and then to stem version

In [7]:
stemmer = PorterStemmer()

In [10]:
# for example
stemmer.stem('Running')

'run'

In [11]:
corpus = []

stopwords_set = set(stopwords.words('english'))

for i in range(len(df)):
  text = df['text'].iloc[i].lower()
  text = text.translate(str.maketrans('', '', string.punctuation)).split()

  # here str.maketrans('abc','xyz')- this will make a=x, b=y and c=z but we want the third parameter passed i.e., string.punctuation - for example sentence like "Hello, world!". Punctuation marks like commas and exclamation points are like little symbols or marks that we use in sentences. The line text.translate(str.maketrans('', '', string.punctuation)) takes a sentence (stored in the variable text) and removes all these punctuation marks from it, leaving just the letters and spaces = "Hello world"
  #  we've removed the punctuation, we have a string with just letters and spaces, like "Hello world". The .split() part takes this string and breaks it into separate words. It's like cutting up a sentence into individual pieces, each piece being a word. So, "Hello world" becomes a list with two pieces: ["Hello", "world"]

  text = [stemmer.stem(word) for word in text if word not in stopwords_set] # if the word not in stopword take that word
  text = ' '.join(text) # join by white space instead of combining 2 words

  corpus.append(text)

In [15]:
df.text.iloc[0]

"Subject: enron methanol ; meter # : 988291this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminaryflow data provided by daren } .please override pop ' s daily volume { presently zero } to reflect dailyactivity you can obtain from gas control .this change is needed asap for economics purposes ."

In [16]:
corpus[0]

'subject enron methanol meter 988291thi follow note gave monday 4 3 00 preliminaryflow data provid daren pleas overrid pop daili volum present zero reflect dailyact obtain ga control chang need asap econom purpos'

In [18]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(corpus).toarray()
y = df.label_num

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [19]:
X[0] # sucessfully we vectorized it as we can see the array of numbers

array([1, 0, 0, ..., 0, 0, 0])

Model building - randomforest classifier

In [20]:
clf = RandomForestClassifier(n_jobs = -1) # -1 instructs to use all cores of system
clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1)

In [21]:
clf.score(X_test, y_test)

0.9739130434782609

checking if the mail is spam or not

In [35]:
email_to_classify = df.text.values[10] # 10th email
email_to_classify

"Subject: vocable % rnd - word asceticismvcsc - brand new stock for your attentionvocalscape inc - the stock symbol is : vcscvcsc will be our top stock pick for the month of april - stock expected tobounce to 12 cents levelthe stock hit its all time low and will bounce backstock is going to explode in next 5 days - watch it soarwatch the stock go crazy this and next week .breaking news - vocalscape inc . announces agreement to resell mix networkservicescurrent price : $ 0 . 025we expect projected speculative price in next 5 days : $ 0 . 12we expect projected speculative price in next 15 days : $ 0 . 15vocalscape networks inc . is building a company that ' s revolutionizing thetelecommunications industry with the most affordable phone systems , hardware ,online software , and rates in canada and the us . vocalscape , a company withglobal reach , is receiving international attention for the development of voiceover ip ( voip ) application solutions , including the award - winning eyefont

In [36]:
email_text = email_to_classify.lower().translate(str.maketrans('', '', string.punctuation)).split()
email_text = [stemmer.stem(word) for word in text if word not in stopwords_set]
email_text = ' '.join(email_text)

email_corpus = [email_text]
email_corpus

['u b j e c   p r   n l n   b n k   l e r e r   v l u   c z e n r   b n k   e b e r   u e   c n c e r n   f e   n e g r   n l n   b n k   c u n   u   f l l w   w r n   e g   c e   e n   c z e n r   b n k   c c u n   n f r   n e e   u p   p r   c n n u   c   p r e c   c c u n   r e u c   n n c   f r u   w e b   c u l   p l e   k e   5   1 0   n u   n l n   e x p e r   r e n e w   r e c r   r u n   f u u r   p r b l e   n l n   e r v c   h w e v   f l u r   c n f r   r e c r     r e u l   c c u n   u p e n   c n f r   c c u n   r e c r   n e r n e   b n k   e r v c   n e r r u p   c n n u   n r l   c n f r   b n k   c c u n   r e c r   p l e   c l c k   n e   e   l   e n   b e h l f   n l n   b n k   c u n   n l n   b n k   c c u n   c h r e r r   n e   e g   p p l     g n r   e g   h n k   e   c z e n r   f n n c   g r u p']

In [37]:
X_email = vectorizer.transform(email_corpus) # note here do not call fit.transform as it will fit on the already build model

In [38]:
clf.predict(X_email) , df.label_num[10]

(array([1]), 1)

In [39]:
# spam is represented as spam correctly

In [47]:
def classify_email(df, index, vectorizer, clf):
    email_to_classify = df.text.values[index]
    email_text = email_to_classify.lower().translate(str.maketrans('', '', string.punctuation)).split()

    stopwords_set = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    email_text = [stemmer.stem(word) for word in email_text if word not in stopwords_set]

    email_text = ' '.join(email_text)
    email_corpus = [email_text]

    X_email = vectorizer.transform(email_corpus)

    prediction = clf.predict(X_email)
    actual_label = df.label_num[index]

    return prediction[0], actual_label

In [51]:
classify_email(df, 10, vectorizer, clf)

(1, 1)